Steps to be followed:

1. calculate missing data per column to figure if it can be used - done
2. according to bibliography and columns, figure out those to remain - done
3. normalization? do we need to?
4. outliers? - done
5. convert everything into numerical and normalized if needs be
6. separate into training, test, evaluation


In [ ]:
import pandas as pd 
import pathlib

#create dataframe from data csv file as df
df = pd.read_csv("db_measurements_v2.1.0.csv") 

In [6]:
#cell to find percentage of NaNs per column, types it in txt file

#create percentages
size = df['index'].size + 1
nan_array = df.isnull().sum() / size * 100 #creates a series of the percentages

#store in file
nan_array_string = ["%.2f" % i for i in nan_array] #turns percentages into strings

data = {df.columns[col]: nan_array_string[col] for col in range(nan_array.size)} #makes dict and dataframe
nan_df = pd.DataFrame(data.items())

path = str(pathlib.Path().resolve()) + '\data.txt' #stores in file
nan_df.to_csv(path, header=None, index = None, sep = ' ')

Picking between columns to be used

Simply by sorting the percentages of NaN cells it is found that only a few of the columns are filled to more than 50%.
Following that, according to the bibliography, most of the parameters usually taken into consideration during experiments are:

met
set
ta
rh
season
gender
age
probably air velocity (vel)
and we're looking for thermal sensation
we'll also be using the acceptability, preferance and comfort where they exist i think but that would require dropping the rows so we'll have to check

Below is the code to sort the NaN array and there's a file where it gets stored sorted.


In [58]:
#sort through nan series and cut all percentages above 50%

nan_array_sorted = nan_array.sort_values(ascending=True) #sorts throught the series 
nan_array_sorted = nan_array_sorted[nan_array_sorted<50.0] #only keeps columns with below 50% NaN cells 

path = str(pathlib.Path().resolve()) + '\data_sorted.txt' #stores file for future use
nan_array_sorted.to_csv(path, header = None, sep = ' ')

Next step is outliers but I'm pretty certain it's not necessary here since those are experiment data and thus already checked and inserted by hand. That said, i'm still running it just to be sure.

According to zscores, on a database including only ta, rh, vel and met there's 5382 outliers. 

In [ ]:
#keeping only a few of the columns for the test in df_outliers dataframe
#TODO: figure out which exact columns to keep
df_outliers  = df[['ta', 'rh', 'vel', 'met']]
print(df_outliers)


In [ ]:
#different outlier methods
#z-scores - using scipy, applying for each column and finding 3 or more deviation cells
import scipy.stats as stats
import math

df_zscore = stats.zscore(df_outliers, nan_policy = 'omit')
print(df_zscore)

counter = 0
for cell in df_zscore['rh']: 
    if (not math.isnan(cell)) and (cell>3 or cell<-3):
        if counter<45: print (cell)
        counter+=1
print (counter)

In [ ]:
#different outlier methods 
#iqr - again just checking, going to fix code here 
import numpy as np 
import math

def iqr_func(column):
    q75, q25 = np.percentile(column, [75 ,25])
    iqr = q75 - q25
    valid = iqr*2.0
    counter = 0
    for cell in column:
        if  (not math.isnan(cell)) and (cell>q75+valid or cell<q25-valid): 
            counter+=1

    return counter

for col in df_outliers.columns: 
    counter = iqr_func(df_outliers[col])
    print(counter)

#supposedly better since it doesn't expect some sort of normality
#IN THIS CASE THO? 

In [ ]:
#different outlier methods
#TODO: plotting for a second - pillow doesn't work actually interested in checking this out
import seaborn as sns 
sns.boxplot(df_outliers['ta'])

In [ ]:
#different outlier methods 
#DBSCAN hopefully - isolation tree? 


To figure out if I need to normalize I need to know what model I will be using. That said, it's probably useful to leave space for this step and then see what to do. So that's a reminder. 